In [ ]:
# !pip install import-ipynb
import import_ipynb
from environment import TicTacToe3D

import random
import numpy as np

In [ ]:
def calculateAllWin():
    # oneD = [(1,0,0), (0,1,0), (0,0,1)]
    # twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
    # threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
    # win_masks = oneD + twoD + threeD

    # all_wins = []
    # tmp = np.zeros((4, 4, 4), dtype=int)
    # for i in range(4):
    #     for j in range(4):
    #         for k in range(4):
    #             for dx, dy, dz in win_masks:
    #                 for l in range(4):
    #                     if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4:
    #                         break
    #                     tmp[i + dx * l, j + dy * l, k + dz * l] = 1
    #                 else:
    #                     all_wins.append(tmp.copy())
    #                 tmp.fill(0)
    # return all_wins

    oneD = [(1,0,0), (0,1,0), (0,0,1)]
    twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
    threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
    win_masks = oneD + twoD + threeD

    all_wins = []
    tmp = np.zeros((4, 4, 4), dtype=int)
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for dx, dy, dz in win_masks:
                    for l in range(4):
                        if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4:
                            break
                        tmp[i + dx * l, j + dy * l, k + dz * l] = 1
                    else:
                        all_wins.append(tmp.copy())
                    tmp.fill(0)

    return all_wins

In [ ]:
class DumbAgent:
    def __init__(self, depth=3, logging=False):
        self.all_wins = calculateAllWin()
        self.depth = depth
        self.logging = logging

    def count_wins(self, board, player):
        c = 0
        for win in self.all_wins:
            if np.sum(win * board) == 4 * player:
                c += 1
        return c

    def heuristic(self, board, player, count_player_wins):
        opposing_filled = board.copy()
        opposing_filled[opposing_filled == 0] = -player
        margin =  count_player_wins - self.count_wins(opposing_filled, -player)

        threat_count = 0
        for win in self.all_wins:
            line = board * win
            if np.count_nonzero(line == player) == 3 and np.count_nonzero(line == 0) == 1:
                threat_count += 100  # Higher weight for creating threats (almost winning lines)
            elif np.count_nonzero(line == player) == 2 and np.count_nonzero(line == 0) == 2:
                threat_count += 50  # Lower weight for potential threats

        return margin + threat_count
    
    def minimax(self, board, player, depth, alpha, beta, minimizingPlayer):
        game = TicTacToe3D().loadState(board)
        result = game.check()

        if result == player:
            return 1000 + depth
        elif result == -player:
            return -1000 - depth
        if depth == 0 or np.all(game.heights == 4):  # Check for a full board
            return self.heuristic(board, player, self.count_wins(board, player))
        
        best_value = float('-inf') if not minimizingPlayer else float('inf')
        compare = max if not minimizingPlayer else min

        for i in range(4):
            for j in range(4):
                if game.heights[i, j] < 4:
                    game.move(i, j, player)  # Move for the current context player
                    value = self.minimax(game.board, -player, depth - 1, alpha, beta, not minimizingPlayer)
                    game.loadState(board)

                    best_value = compare(best_value, value)
                    if not minimizingPlayer:
                        alpha = max(alpha, best_value)
                    else:
                        beta = min(beta, best_value)

                    if beta <= alpha:
                        break
            if beta <= alpha:
                break

        return best_value

    def findBestMove(self, board, possible_move, player):
        game = TicTacToe3D()
        game.loadState(board)

        best = float('-inf')
        bestMove = None
        
        if len(possible_move) == 0:
            return None
        
        # Check the initial block necessity
        initial_blocks = set()
        for win in self.all_wins:
            line = board * win
            if np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1:
                initial_blocks.add(str(win.nonzero()))  # Store win patterns that are already blocking moves
        bestMove = random.choice(possible_move)
                        
        for i, j in possible_move:
            game.move(i, j, player)
            new_board = game.board.copy()
            
            for win in self.all_wins:
                line = new_board * win
                if np.count_nonzero(line == player) == 4:
                    return (i, j)
                elif np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1 and str(win.nonzero()) not in initial_blocks:
                    return (i, j)
            
            moveVal = self.minimax(new_board, -player, self.depth, float('-inf'), float('inf'), True)
            game.loadState(board)
            if moveVal > best:
                best = moveVal
                bestMove = (i, j)
                if self.logging:
                    print(f"Move value for ({i}, {j}): {moveVal}")  # To observe the value of each move
        return bestMove

In [ ]:
# import numpy as np

# # dummy_board = np.zeros((4, 4, 4), dtype=int)
# # floor, row, col
# # dummy_board[0, 1, 0] = 1
# # dummy_board[0, 1, 1] = 1
# # dummy_board[0, 2, 2] = 1
# # dummy_board[0, 3, 2] = 1

# # dummy_board[0, 0, 0] = -1
# # dummy_board[1, 0, 0] = 1
# # dummy_board[2, 0, 0] = 1
# # dummy_board[3, 0, 0] = 1

# # dummy_board[1, 0, 0] = -1
# # dummy_board[2, 0, 0] = -1

# dummy_game = TicTacToe3D()
# dummy_game.move(0, 0, 1)
# dummy_game.move(0, 3, -1)

# dummy_game.move(1, 0, 1)
# dummy_game.move(3, 0, -1)

# dummy_game.move(1, 2, 1)
# dummy_game.move(1, 1, -1)

# dummy_game.move(0, 2, 1)
# dummy_game.move(1, 1, -1)

# dummy_game.move(2, 2, 1)
# dummy_game.move(3, 2, -1)

# dummy_game.move(1, 2, 1)
# dummy_game.move(0, 0, -1)

# dummy_game.move(3, 2, 1)
# dummy_game.move(0, 0, -1)

# dummy_game.move(0, 2, 1)

# # print(dummy_game.board[::-1])

# agent = DumbAgent(logging=True, depth=3)
# x = agent.findBestMove(dummy_game.board, -1)
# print(x)

In [ ]:
    # def heuristic(self, board, player):
    #     immediate_win_score = 100
    #     block_opponent_score = 50
    #     heuristic_value = 0

    #     for win in self.all_wins:
    #         line = board.copy() * win.copy()
    #         if np.count_nonzero(line == player) == 4:
    #             heuristic_value += immediate_win_score
    #         elif np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1:
    #             heuristic_value += block_opponent_score

    #     return heuristic_value